In [2]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import steplib as msl
%matplotlib inline

In [3]:
PATH = "Sheep"

all_files = os.listdir(PATH)

print(all_files) 


['GroundMax.csv', 'GroundPT.csv', 'GroundPT2.csv', 'outputMax.csv', 'outputPT.csv', 'stateMax.csv', 'statePT.csv']


In [4]:
import numpy as np
end = 60000

def getSteps(file, window_size=20, threshold=1.15):
    file_data = pd.read_csv("Sheep/" + file)[0:end]
    expected = np.where(file_data["gyrx"] != 0)[0]
    data1 = file_data["accx"] ** 2 + file_data["accy"] ** 2 + file_data["accz"] ** 2
    accelerometer = data1.apply(lambda x: x ** 0.5)
    data2 = msl.apply_filter(accelerometer, [0.0167855014066363,	0.0470217363830679,	0.121859539593060,	0.198782647391950,	0.231101150450572,	0.198782647391950,	0.121859539593060,	0.0470217363830679,	0.0167855014066363])
    data1 = msl.maximum_difference(data2, window_size)
    data1 = msl.detect_outliers(data1, threshold)
    data1 = msl.post_processing(data1, 250, 100)
    calculated = np.where(data1 == 1)[0]


    return calculated, expected , data2

In [5]:
def chart(data_in):
    (calculated, expected, accelerometer) = data_in
    x=range(len(accelerometer))
    y=accelerometer
    plt.figure(figsize=(10,5))
    plt.plot(x,y, color='red', marker='o')
    plt.xlabel('index')
    plt.ylabel('magnitude')
    #show x label every 800
    plt.xticks(x[::500])
    plt.xticks(rotation=90)
    #add rectangles
    for e in expected:
        plt.axvline(x =e, color ='g', label ='axvline - full height', ymin=0.5, alpha=0.8)
    for c in calculated:
        plt.axvline(x =c, color ='b', label ='axvline - full height', ymax=0.5, alpha=0.8)
    plt.show()

In [6]:
differences = []
i = 0
for file in all_files:
    (calculated1, expected1, _) = getSteps(file)
    differences.append(len(expected1) - len(calculated1))
    print(i, len(expected1), len(calculated1), file)
    i += 1
print(differences) 

KeyError: 'gyrx'

In [8]:
%matplotlib qt
chart(getSteps(all_files[0]))

KeyError: 'gyrx'

In [ ]:
#TODO: new datasets
#TODO: new filter
#TODO   

In [7]:
from scipy.optimize import minimize

def objective(x):
    result = getSteps(all_files[0], int(x[0]), x[1])
    return abs(len(result[1]) - len(result[0]))


x0 = np.array([20, 1.15])
res = minimize(objective, x0, method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
print(res.x)

KeyError: 'gyrx'